In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import  torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
random.seed(1024)

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor
ByteTensor = torch.ByteTensor



In [2]:
nltk.corpus.gutenberg.fileids()


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
nltk.corpus.gutenberg.sents('melville-moby_dick.txt')[:3]


[['[', 'Moby', 'Dick', 'by', 'Herman', 'Melville', '1851', ']'],
 ['ETYMOLOGY', '.'],
 ['(',
  'Supplied',
  'by',
  'a',
  'Late',
  'Consumptive',
  'Usher',
  'to',
  'a',
  'Grammar',
  'School',
  ')']]

In [4]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:100] # sampling sentences for test
corpus = [  [word.lower() for word in sent] for sent in corpus  ]
for i in range(3):
    print(corpus[i])


['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']']
['etymology', '.']
['(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to', 'a', 'grammar', 'school', ')']


In [5]:
flatten = lambda l: [item    for sublist in l     for item in sublist]
#flattern is a functionword_count = Counter(flatten(corpus))
#faltten是匿名函数，三维变为一维

print(len(flatten(corpus)))

word_count = Counter(flatten(corpus))
print(len(word_count))
#共有592个不同的单词/word
# print(word_count)
# Counter({',': 96, '.': 66, 'the': 58, 'of': 36,  'man': 1})
border = int(len(word_count)*0.01) 
print(border)



1463
592
5


In [6]:
# print(  type(word_count.most_common()  ))
 # [(',', 96),('.', 66),('the', 58),('of', 36), ('and', 35)......,('man', 1)]

stopwords_first = word_count.most_common()[:border] 
stopwords_last = list(reversed(word_count.most_common()))[:border]
stopwords = stopwords_first + stopwords_last
print(stopwords)
stopwords = [ s[0] for s in stopwords]
stopwords
#stopwords 是corpus中个数最多和 最少的几个单词


[(',', 96), ('.', 66), ('the', 58), ('of', 36), ('and', 35), ('man', 1), ('artificial', 1), ('civitas', 1), ('--(', 1), ('state', 1)]


[',', '.', 'the', 'of', 'and', 'man', 'artificial', 'civitas', '--(', 'state']

In [7]:
#vocab为 减去最多和最少频率的单词 之后的list
vocab= list(  set(flatten(corpus))-set(stopwords)   )
vocab.append('<UNK>')
# print(vocab)
print(len(set(flatten(corpus))), len(vocab))

592 583


In [8]:
#最后得到的  word2index  为除去减去最多和最少频率的单词之后的  {'UNK':0,单词：1，单词：2...}
# 最后得到的  index2word  为除去减去最多和最少频率的单词之后的 {0:'UNK',1:单词，2:单词....}

word2index = {'<UNK>':0}
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
index2word = { v:k for k,v in  word2index.items()}

In [9]:
# print(corpus)
'''
[   [   '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']'], 
    ['etymology', '.'], 
    ['(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to', 'a', 'grammar', 'school', ')'],
    ['the', 'pale', 'usher', '--', 'threadbare', 'in', 'coat', ',', 'see', 'him', 'now', '.']]
'''
#从corpus中取wind_size=3 的滑动窗口
WINDOW_SIZE = 3
windows = flatten(  [list(    nltk.ngrams(  ['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1  )                ) 
                          for c in corpus  ]   
                 )
# print(windows)
'''
[		('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by'), 
		('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman'), 
		('<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville'), 
		('[', 'moby', 'dick', 'by', 'herman', 'melville', '1851'), 
		('moby', 'dick', 'by', 'herman', 'melville', '1851', ']'), 
		('dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>'), 
		('by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>'), 
		('herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>'), 
		
		('<DUMMY>', '<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>'), 
		('<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>', '<DUMMY>'), 
		
		('<DUMMY>', '<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a'), 
		('<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a', 'late'), 
		('<DUMMY>', '(', 'supplied', 'by', 'a', 'late', 'consumptive'), 
		('(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher'), 
		('supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to'), 
		('by', 'a', 'late', 'consumptive', 'usher', 'to', 'a'), 
		('a', 'late', 'consumptive', 'usher', 'to', 'a', 'grammar'), 
		('late', 'consumptive', 'usher', 'to', 'a', 'grammar', 'school'), 
		('consumptive', 'usher', 'to', 'a', 'grammar', 'school', ')'), 
		('usher', 'to', 'a', 'grammar', 'school', ')', '<DUMMY>'), 
		('to', 'a', 'grammar', 'school', ')', '<DUMMY>', '<DUMMY>'), 
		('a', 'grammar', 'school', ')', '<DUMMY>', '<DUMMY>', '<DUMMY>')
		............................]

'''
print(windows[0])
print(windows[1])


('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by')
('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman')


In [10]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': 
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))
print(train_data[:10])
print(word2index['['])


    

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('moby', '['), ('moby', 'dick'), ('moby', 'by'), ('moby', 'herman'), ('dick', '['), ('dick', 'moby'), ('dick', 'by')]
475


In [11]:
       
     
def prepare_sequence(seq, word2index):
    idxs = list(   map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq)         ) 
    return Variable(LongTensor(idxs))


def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]])   if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))   
        


In [12]:
X_p = []
y_p = []

for tr in train_data[:2]:
    print("X_p= ",prepare_word(tr[0],word2index),"       y_p=",prepare_word(tr[1],word2index))
    print("X_p= ",prepare_word(tr[0],word2index).view(1,-1),"     y_p=",prepare_word(tr[1],word2index).view(1,-1),'\n')
    
#view相当于reshape               tensor的数据个数是6个，如果view（1，-1），我们就可以根据tensor的元素个数推断出-1代表6，维度为1*6
for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))

X_p=  tensor([ 475])        y_p= tensor([ 288])
X_p=  tensor([[ 475]])      y_p= tensor([[ 288]]) 

X_p=  tensor([ 475])        y_p= tensor([ 441])
X_p=  tensor([[ 475]])      y_p= tensor([[ 441]]) 



In [17]:
print(X_p[:4][0].size())
print(X_p[:4])
print(y_p[:4])
train_data = list(zip(X_p, y_p))
print(train_data[:4])
''''
[	(tensor([[ 100]]), tensor([[ 345]])),           ========>('[', 'moby')
	(tensor([[ 100]]), tensor([[ 245]])),           ========> ('[', 'dick')
	(tensor([[ 100]]), tensor([[ 571]])),           ========>  ('[', 'by'),
	(tensor([[ 345]]), tensor([[ 100]]))            ========>  ('moby', '[')
]
'''





torch.Size([1, 1])
[tensor([[ 475]]), tensor([[ 475]]), tensor([[ 475]]), tensor([[ 288]])]
[tensor([[ 288]]), tensor([[ 441]]), tensor([[ 481]]), tensor([[ 475]])]
[(tensor([[ 475]]), tensor([[ 288]])), (tensor([[ 475]]), tensor([[ 441]])), (tensor([[ 475]]), tensor([[ 481]])), (tensor([[ 288]]), tensor([[ 475]]))]


"'\n[\t(tensor([[ 100]]), tensor([[ 345]])),           ========>('[', 'moby')\n\t(tensor([[ 100]]), tensor([[ 245]])),           ========> ('[', 'dick')\n\t(tensor([[ 100]]), tensor([[ 571]])),           ========>  ('[', 'by'),\n\t(tensor([[ 345]]), tensor([[ 100]]))            ========>  ('moby', '[')\n]\n"

In [20]:
len(train_data)

7606

In [21]:
print(len(word2index))


583


In [22]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch
        
        
        
        
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)   # D x V
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words,target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D            V_c = (d*v) * (v * b) = (d *b)
        target_embeds = self.embedding_u(target_words) # B x 1 x D            U_o = (d*v) * (v * b) = (d *b)
        outer_embeds = self.embedding_u(outer_words) # B x V x D              U_w = (d*v) * (v * b) = (d * b)
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)          # (V_c.T * U_o) =  (b*d) *(d*b) = (b*b)
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)      # (V_c.T * V_w)=   (b*d) * (d*b) =(b*b)
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 
    
    
    



EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100


model = Skipgram(len(word2index), EMBEDDING_SIZE)
optimizer = optim.Adam(model.parameters(), lr=0.01)
losses = []



        

for epoch in range(EPOCH):
    # a=getBatch(BATCH_SIZE,train_data)
    # print(type(a) )  #<class 'generator'>
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        # if i==2:
        #     print("\n1",batch)    #batch是元组组成的数组
            # print("\n2",*batch)  #  *batch是元组
        '''
           [  (tensor([[ 341]]), tensor([[ 26]])), 
                (tensor([[ 215]]), tensor([[ 429]])), 
                (tensor([[ 361]]), tensor([[ 508]]))] 
                
            (tensor([[ 341]]), tensor([[ 26]])) 
            (tensor([[ 215]]), tensor([[ 429]])) 
            (tensor([[ 361]]), tensor([[ 508]]))
         '''
        inputs, targets = zip(*batch) 
        # if i==2:
        #     print(inputs)
        #         (tensor([[ 47]]),    tensor([[ 0]]),   tensor([[ 163]]) 
        inputs = torch.cat(inputs) # B x 1
        # if i==2:
        #     print('\n',inputs)
       #         tensor ( [  [ 47], [0], [ 163]  ] ) 
        targets = torch.cat(targets) # B x 1
        
        
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab))  # B x V
        
        
        
        model.zero_grad()
        loss = model(inputs, targets, vocabs)
        #反向传播
        loss.backward()
        #优化梯度   w = w + lr * grad
        optimizer.step()
    
   
        losses.append(loss.data.tolist())

    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []

Epoch : 0, mean_loss : 1.93


Epoch : 10, mean_loss : 0.00


Epoch : 20, mean_loss : 0.00


Epoch : 30, mean_loss : 0.00


Epoch : 40, mean_loss : 0.00


Epoch : 50, mean_loss : 0.00


Epoch : 60, mean_loss : 0.00


Epoch : 70, mean_loss : 0.00


Epoch : 80, mean_loss : 0.00


Epoch : 90, mean_loss : 0.00


In [26]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: continue
        vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0] 
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:50]   # sort by similarity

test = random.choice(list(vocab))
print(test)
word_similarity(test, vocab)

plainly


[['these', 0.5557811856269836],
 ['splintered', 0.5465330481529236],
 ['monstrous', 0.5317400097846985],
 ['down', 0.5246033668518066],
 ['random', 0.46905726194381714],
 ['ruin', 0.4616011381149292],
 ['hwal', 0.4541151821613312],
 ['learned', 0.4469209909439087],
 ['dart', 0.4407564699649811],
 ['was', 0.42691487073898315],
 ['piercing', 0.4191505014896393],
 ['lucian', 0.41689443588256836],
 ['were', 0.3968399167060852],
 ['one', 0.38981321454048157],
 ['to', 0.38828083872795105],
 ['deliver', 0.38629209995269775],
 ['mockingly', 0.38288551568984985],
 ['consumptive', 0.3825888931751251],
 ['god', 0.35731345415115356],
 ['rolling', 0.35263940691947937],
 ['sit', 0.34989187121391296],
 ['mast', 0.34026435017585754],
 ['burrower', 0.33982089161872864],
 ['whatever', 0.3328428566455841],
 ['parmacetti', 0.33131229877471924],
 ['pampered', 0.32948189973831177],
 ['quid', 0.3240537941455841],
 ['but', 0.3191649615764618],
 ['reminded', 0.3175491392612457],
 ['bruise', 0.2970965504646301]